In [9]:
import salabim_plus as sim_plus
import salabim as sim
import multiplexer_attempt1 as multiplexer_attempt
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 100
 
with open('data/output_firsttry_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
    env = sim_plus.EnvironmentPlus(trace=out)
    
    # step 2: create the machines in the simulation environment
    assembly_bench = sim_plus.Machine(var_name='assembly_bench', env=env)
    quality_bench = sim_plus.Machine(var_name='quality_bench', env=env)
    
    machine = sim_plus.Machine(var_name='machine', env=env)
    
    # step 4: create the workers in the simulation environment
    assembler = sim_plus.Worker(var_name='assember', env=env, capacity=1)
    technician = sim_plus.Worker(var_name='technician', env=env, capacity=1)
    qual_inspector = sim_plus.Worker(var_name='qual_inspector', env=env, capacity=1)
    
    # step 5: make the shift schedules for the workers
    shift_schedule = misc_tools.make_shifts(shift_duration=8*60,
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    qual_inspector_shift = sim_plus.ShiftController(worker=qual_inspector,
                                                   env=env,
                                                   start_time=480,
                                                   shifts=shift_schedule,
                                                   shift_type='pattern')
    
    # step 6: 
    multiplexer_steps_func = multiplexer_attempt.create_routing
    
    #step 7:
    multiplexer_attempt_gen = sim_plus.EntityGenerator(var_name='multiplexer_attempt',
                                                       steps_func=multiplexer_steps_func,
                                                       env=env,
                                                       arrival_type='ordered')
    
    # step 8:
    multiplexer_attempt_kanban = sim_plus.Kanban(var_name='multiplexer_attempt',
                                    env=env, kanban_attr=multiplexer_attempt.create_kanban_attrs(env.objs))
    
    # step 9:
    multiplexer_attempt_gen.main_exit = multiplexer_attempt_kanban
    
    #step 10:
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    qual_inspector_shift.activate(process='work')
    
    #step 11:
    env.run(till=runtime)